In [ ]:
import numpy as np
from math import cos, sin, pi
import matplotlib.pyplot as plt
%matplotlib inline

""" Kareem's Note:

Here I define a set of (N) points as a 2xN matrix where each point is a column vector
    eg) (x, y) -> np.array([[x],[y]])

This definition makes it very easy to apply a transformation: just left-multiply the points with the desired transform.
To transform multiple groups of points with the same transform you can use the helper function 
transform_points(Transform, group1, group2, ...)

To plotting functions take an arbitrary number of these 2D matrices and plots each as a separate group
Each group of points will be drawn by connecting consecutive points. 
As a result, to draw a vector you need 2 points, thus the vector_to_points() function.

"""

def generate_unit_circle(steps=40):
    points = []
    theta  = 0
    delta  = 2*pi/steps
    for step in range(steps):
        points.append(np.array([[cos(theta)],[sin(theta)]]))
        theta += delta
    points.append(points[0])
    return np.hstack(points)

def vector_to_points(vector):
    return np.hstack((vector, np.zeros(shape=vector.shape)))

def get_basis_vectors():
    return vector_to_points(np.array([[1],[0]])), vector_to_points(np.array([[0],[1]]))

def plot_points(*point_sets, title="", labels=[], figsize=8, scale=4):
    fig = plt.figure(figsize=(figsize, figsize))
    labels = labels or [f"argument_{i}" for i in range(len(point_sets))]
    for points, label in zip(point_sets, labels):
        plt.plot(*points, label=label)
    plt.xlim([-scale, scale])
    plt.ylim([-scale, scale])
    plt.legend(prop={"size":16})
    plt.title(title, fontsize=18)
    plt.grid(axis='both')
    plt.show()
       
def plot_multiple_points(*point_sets, labels=[], figsize=5, scale=5):
    labels = labels or [f"argument_{i}" for i in range(len(point_sets))]
    for points, label in zip(point_sets, labels):
        fig = plt.figure(figsize=(figsize, figsize))
        #plt.rc('text', usetex=True)
        #https://het.as.utexas.edu/HET/Software/Matplotlib/users/usetex.html
        plt.plot(*points, label=label)
        plt.xlim([-scale, scale])
        plt.ylim([-scale, scale])
        plt.legend()
        plt.show()
        
def transform_points(transform, *args):
    return [transform @ points for points in args]

def get_col2D(M, col):
    return M[:,col:col+1]

# Uncomment this line if latex doesn't work for you. Note that this breaks latex on JupyterHub.
#plt.rcParams.update({
#    "text.usetex": True,
#})

In [ ]:
# PART A

# Define transformation matrix
A = np.array([
    # ----- YOUR CODE HERE ----- #
    [0,-1],
    [3,0]
])

circle = generate_unit_circle()
plot_points(A @ circle, title="AS", labels=["A circle"], scale=5)

In [ ]:
# PARTS B-D

# Compute the SVD
U,s,VT = np.linalg.svd(A)
E = np.diag(s) # this only works for square matrices. Other matrices will need to pad with zeros

print(f"U: \n{U}\n")
print(f"Sigma: \n{E}\n")
print(f"V.T: \n{VT}\n")

# 
plot_basis_vectors = False # Whether to plot e_1 and e_2 instead of v_1 and v_2

# Create unit circle and get vectors for plotting
circle = generate_unit_circle()
e1, e2 = get_basis_vectors()
v1     = vector_to_points(get_col2D(VT.T, 0))
v2     = vector_to_points(get_col2D(VT.T, 1))
points_basis = (circle, e1, e2)
points_vcols = (circle, v1, v2)
points = points_basis if plot_basis_vectors else points_vcols

# Transformed points (note that @ is conventionally used as the dot product operator in python)
points_vt = transform_points(VT, *points)
points_e = transform_points(E, *points)
points_evt = transform_points(E @ VT, *points)
points_uevt = transform_points(U @ E @ VT, *points)

# Plot transformations
if plot_basis_vectors:
    plot_points(*points     , title=r"$Circle$"                , labels=[r"$Circle$"                , r"$\vec{e}_1$", r"$\vec{e}_1$"])
    plot_points(*points_vt  , title=r"$V^\top Circle$"         , labels=[r"$V^\top Circle$"         , r"$V^\top\vec{e}_1$" , r"$V^\top\vec{e}_2$" ])
    plot_points(*points_evt , title=r"$\Sigma V^\top Circle$"  , labels=[r"$\Sigma V^\top Circle$"  , r"$\Sigma V^\top\vec{e}_1$", r"$\Sigma V^\top\vec{e}_2$"])
    plot_points(*points_uevt, title=r"$U \Sigma V^\top Circle$", labels=[r"$U \Sigma V^\top Circle$", r"$U\Sigma V^\top\vec{e}_1$", r"$U\Sigma V^\top\vec{e}_2$"])
else:
    plot_points(*points     , title=r"$Circle$"                , labels=[r"$Circle$"                , r"$\vec{v}_1$", r"$\vec{v}_1$"])
    plot_points(*points_vt  , title=r"$V^\top Circle$"         , labels=[r"$V^\top Circle$"         , r"$V^\top\vec{v}_1$" , r"$V^\top\vec{v}_2$" ])
    plot_points(*points_evt , title=r"$\Sigma V^\top Circle$"  , labels=[r"$\Sigma V^\top Circle$"  , r"$\Sigma V^\top\vec{v}_1$", r"$\Sigma V^\top\vec{v}_2$"])
    plot_points(*points_uevt, title=r"$U \Sigma V^\top Circle$", labels=[r"$U \Sigma V^\top Circle$", r"$U\Sigma V^\top\vec{v}_1$", r"$U\Sigma V^\top\vec{v}_2$"])
                
# FIXME rename labels in intermediate steps

In [ ]:
# PART F

# Plot semi-major, semi-minor axis of ellipse

# Get Columns of U (while keeping 2D shape)
u_1 = U[:,0:1] 
u_2 = U[:,1:2] 

# Transform 
AS = A @ circle
su_1 = s[0]*u_1
su_2 = s[1]*u_2

show_basis_vectors = False

# include transformed basis vectors version
if show_basis_vectors:
    _points = (*transform_points(A, *points_basis), vector_to_points(su_1), vector_to_points(su_2))
    _labels = ["$ACircle$",r"$A\vec{e}_1$", r"$A\vec{e}_2$", r"$\sigma_1\vec{u}_1$", r"$\sigma_2\vec{u}_2$"]

# don't include transformed basis vectors
else:
    _points = (*transform_points(A, circle), vector_to_points(su_1), vector_to_points(su_2))
    _labels = ["$ACircle$",r"$\sigma_1\vec{u}_1$", r"$\sigma_2\vec{u}_2$"]

# Plot
plot_points(*_points, title=r"$ACircle$", labels=_labels)
